In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from datetime import datetime as dt
from math import sqrt, isnan, sin, cos, asin, pi, atan2
import requests
import gzip
from functools import reduce
from scipy import interpolate

import warnings
warnings.catch_warnings()
warnings.simplefilter("ignore")

In [2]:
def get_NOAA_data():
    
    URL = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
    r = requests.get(URL)
    file_names = pd.read_html(r.text)[0]['Name']
    events_file_names = file_names[file_names.str.contains("details",na=False)]
    noaa_list = []
    
    print("Extracting CSV files...")
    for file in events_file_names:
        full_URL = URL + file
        with gzip.open(requests.get(full_URL, stream=True).raw) as f:
            noaa_list.append(pd.read_csv(f))
        
    df = pd.concat(noaa_list)
    
    print("Completed")
    return df

def pickle_source_data():
    noaa_source_df = get_NOAA_data()
    home_dir = os.getcwd()
    data_dir = os.path.join(home_dir, "Data")
    try:
        os.mkdir(data_dir)
        os.chdir(data_dir)
    except OSError:
        os.chdir(data_dir)
        for file in os.listdir():
            os.remove(file)
    noaa_source_df.to_pickle('noaa_source_data.pkl')
    os.chdir(home_dir)
    return noaa_source_df

NOAA_df = pickle_source_data()
    

Extracting CSV files...
Completed


In [3]:
NOAA_df = pd.read_pickle('Data/noaa_source_data.pkl')

In [4]:
def replace_str2num(x):
    if type(x) == float or type(x) == int:
        return float(x)
    num = 1 if x[:-1] == '' else x[:-1]        
    if x[-1] == 'T':
        return float(num) * 1000000000000
    elif x[-1] == 'B':
        return float(num) * 1000000000
    elif x[-1] == 'M':
        return float(num) * 1000000
    elif x[-1] == 'K' or x[-1] == 'k':
        return float(num) * 1000
    elif x[-1] == 'h' or x[-1] == 'H':
        return float(num) * 100
    elif x[-1] == '?':
        return float(num)
    else:
        return float(x)

def winds(x):
    if x['MAGNITUDE_TYPE'] in ['EG', 'E', 'M', 'ES', 'MG', 'MS']:
        return x['MAGNITUDE']

def hail(x):
    if x['MAGNITUDE_TYPE'] not in ['EG', 'E', 'M', 'ES', 'MG', 'MS']:
        return x['MAGNITUDE']

def missing_swap(df, col1, col2):
    df.loc[~df[col1].isnull() & df[col2].isnull(), col2] = df.loc[~df[col1].isnull() & df[col2].isnull(), col1]
    df.loc[df[col1].isnull() & ~df[col2].isnull(), col1] = df.loc[df[col1].isnull() & ~df[col2].isnull(), col2]
    return df

def calc_duration(x):
    begin_dt = dt.strptime(x['BEGIN_DATE_TIME'], "%d-%b-%y %H:%M:%S")
    end_dt = dt.strptime(x['END_DATE_TIME'], "%d-%b-%y %H:%M:%S")
    difference = end_dt - begin_dt
    difference_days = difference.days + difference.seconds/86400
    return difference_days

def geo_distance(x):
    # Source : https://en.wikipedia.org/wiki/Haversine_formula
    p = pi/180
    lat1 = x['BEGIN_LAT']
    lat2 = x['END_LAT']
    lon1 = x['BEGIN_LON']
    lon2 = x['END_LON']
    R = 6371
    dLat = p * (lat2-lat1)
    dLon = p * (lon2-lon1)
    a = sin(dLat/2) ** 2 + cos(p*lat1) * cos(p*lat2) * sin(dLon/2) ** 2
    return 2 * R * atan2(sqrt(a), sqrt(1-a))

rename_event_dict = {
    'TORNADOES, TSTM WIND, HAIL': 'Tornadoes, Thunderstorm Wind, Hail',
    'THUNDERSTORM WINDS LIGHTNING': 'Thunderstorm Wind, Lightning',
    'THUNDERSTORM WINDS/ FLOOD': 'Thunderstorm Wind, Flood',
    'THUNDERSTORM WINDS/FLOODING': 'Thunderstorm Wind, Flood',
    'THUNDERSTORM WIND/ TREES': 'Thunderstorm Wind, Trees',
    'THUNDERSTORM WIND/ TREE': 'Thunderstorm Wind, Trees',
    'THUNDERSTORM WINDS/HEAVY RAIN': 'Thunderstorm Wind, Heavy Rain',
    'TORNADO/WATERSPOUT': 'Tornado, Waterspout',
    'THUNDERSTORM WINDS FUNNEL CLOU': 'Thunderstorm Wind, Funnel Cloud',
    'THUNDERSTORM WINDS/FLASH FLOOD': 'Thunderstorm Wind, Flash Flood',
    'HAIL/ICY ROADS': 'Hail/ Icy Roads',
    'HAIL FLOODING': 'Hail, Flood',
    'THUNDERSTORM WINDS HEAVY RAIN': 'Thunderstorm Wind, Heavy Rain',
    'Hurricane (Typhoon)': 'Hurricane'
}

timezone_dict = {
    'GST' : ['GST10'],
    'AST' : ['AST-4', 'AST'],
    'EST' : ['EST', 'EST-5', 'ESt', 'EDT'],
    'CST' : ['CST', 'CST-6', 'CSt', 'CSC', 'SCT', 'GMT', 'UNK', 'CDT'],
    'MST' : ['MST', 'MST-7', 'MDT'],
    'PST' : ['PST', 'PST-8', 'PDT'],
    'AKST': ['AKST-9'],
    'HST' : ['HST-10', 'HST'],
    'SST' : ['SST-11', 'SST']
}

azimuth_mapping ={ 'N/A': ['ND', 'EE', 'TO', 'MI', 'M', 'EST', 'EAS', 'TH', 'WES'] }

def dict_mapping(x):
    for key, val in timezone_dict.items():
        if x in val:
            return key
        
def tor_scale(x):
    if type(x)==float or x[-1] == 'U':
        return 0
    else:
        return int(x[-1]) + 1

In [5]:
def impute_NOAA_data(df):
    
    # maginitude converted into other variables
    drop_list = ['EVENT_NARRATIVE', 'EPISODE_NARRATIVE', 'EPISODE_ID', 'MAGNITUDE']
    impute_mean_list = ['BEGIN_LAT', 'END_LAT', 'BEGIN_LON', 'END_LON', 'DAMAGE_CROPS', 'DURATION', 'GEO_DISTANCE']
    impute_event_mean_list = ['DAMAGE_CROPS', 'DURATION', 'GEO_DISTANCE', 'TOR_WIDTH', 'TOR_LENGTH', 'BEGIN_RANGE', \
                              'END_RANGE', 'WIND_SPEED', 'HAIL_SIZE']
    
    # imputing damage columns with 0 for the time-being
    impute_zero_list = ['TOR_WIDTH', 'TOR_LENGTH', 'BEGIN_RANGE', 'END_RANGE', 'WIND_SPEED', 'HAIL_SIZE']
    impute_NA_list = ['TOR_OTHER_CZ_NAME', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_WFO', \
                      'CATEGORY', 'FLOOD_CAUSE', 'SOURCE', 'WFO', 'CZ_NAME', 'DATA_SOURCE', 'STATE', \
                      'STATE_FIPS', 'MAGNITUDE_TYPE', 'BEGIN_AZIMUTH', 'END_AZIMUTH', 'BEGIN_LOCATION', \
                      'END_LOCATION', 'TOR_OTHER_CZ_FIPS']
    
    # Remove rows with no damage info
    df.dropna(subset=['DAMAGE_PROPERTY'], axis=0, inplace=True)
    
    # Removing inconsistencies
    df = missing_swap(df, 'BEGIN_RANGE', 'END_RANGE')
    df = missing_swap(df, 'BEGIN_LAT', 'END_LAT')
    df = missing_swap(df, 'BEGIN_LON', 'END_LON')
    df = missing_swap(df, 'BEGIN_AZIMUTH', 'END_AZIMUTH')
    df = missing_swap(df, 'BEGIN_LOCATION', 'END_LOCATION')
    
    # Damage variables cleaning
    df['DAMAGE_PROPERTY'] = df.DAMAGE_PROPERTY.map(replace_str2num)
    df['DAMAGE_CROPS'] = df.DAMAGE_CROPS.map(replace_str2num)
    
    # Splitting magnitude variable into constituent attributes
    df['WIND_SPEED'] = df.apply(winds, axis = 1)
    df['HAIL_SIZE'] = df.apply(hail, axis = 1)
    
    # Feature cleaning and engineering
    df['DURATION'] = df[['BEGIN_DATE_TIME', 'END_DATE_TIME']].apply(lambda x: calc_duration(x), axis=1)
    df['GEO_DISTANCE'] = df.apply(lambda x: geo_distance(x), axis=1)
    df['EVENT_TYPE'] = df['EVENT_TYPE'].apply(lambda x: rename_event_dict[x] if rename_event_dict.get(x)!= None else x)
    df['COLD_WEATHER_EVENT'] = df['EVENT_TYPE'].str.contains('Hail|Winter|Snow|Chill|Cold|Frost|Freeze|Blizzard|Ice|Avalanche').map({True: 1, False:0})
    df['WINDY_EVENT'] = df['EVENT_TYPE'].str.contains('Wind|Tornado|Thunderstorm|Cloud|Storm').map({True: 1, False:0})
    df['WATER_EVENT'] = df['EVENT_TYPE'].str.contains('Flood|Marine|Rain|Hurricane|Tide|Lake|Seiche|Tsunami|Sleet|Water').map({True: 1, False:0})
    df.loc[:,'CZ_TIMEZONE'] = df.loc[:,'CZ_TIMEZONE'].apply(lambda x: dict_mapping(x))
    df.loc[:,'BEGIN_AZIMUTH'] = df.loc[:,'BEGIN_AZIMUTH'].str.upper().apply(lambda x: dict_mapping(x) if dict_mapping(x) != None else x)
    df.loc[:,'END_AZIMUTH'] = df.loc[:,'END_AZIMUTH'].str.upper().apply(lambda x: dict_mapping(x) if dict_mapping(x) != None else x)
    df.loc[:,'TOR_F_SCALE'] = df.loc[:,'TOR_F_SCALE'].str.upper().apply(lambda x: tor_scale(x))
    
    # Imputing few columns with mean value for each event type
    for col in impute_event_mean_list:
        df[col] = df.groupby("EVENT_TYPE")[col].transform(lambda x: x.fillna(x.mean()))
    
    # Imputing string columns with missing values with N/A
    for col in impute_NA_list:
        df[col] = df[col].astype('str').apply(lambda x: 'N/A' if x=='nan' else x)
        
    # Imputing float columns having missing values with 0.0
    for col in impute_zero_list:
        df[col] = df[col].fillna(0.0)
        
    # Imputing latitude and longitudes with average value
    for col in impute_mean_list:
        df[impute_mean_list] = df[impute_mean_list].transform(lambda x: x.fillna(x.mean()))
        
    # Dropping text and ID columns
    for col in drop_list:
        df.drop(col, axis=1, inplace=True)
    
    return df

imputed_NOAA_df = impute_NOAA_data(NOAA_df.copy())
imputed_NOAA_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EVENT_ID,STATE,STATE_FIPS,YEAR,...,END_LAT,END_LON,DATA_SOURCE,WIND_SPEED,HAIL_SIZE,DURATION,GEO_DISTANCE,COLD_WEATHER_EVENT,WINDY_EVENT,WATER_EVENT
0,195004,28,1445,195004,28,1445,10096222,OKLAHOMA,40.0,1950,...,35.17,-99.20,PUB,59.0,0.0,0.0,5.559746,0,1,0
1,195004,29,1530,195004,29,1530,10120412,TEXAS,48.0,1950,...,31.73,-98.60,PUB,59.0,0.0,0.0,18.903138,0,1,0
2,195007,5,1800,195007,5,1800,10104927,PENNSYLVANIA,42.0,1950,...,40.65,-75.47,PUB,59.0,0.0,0.0,20.916105,0,1,0
3,195007,5,1830,195007,5,1830,10104928,PENNSYLVANIA,42.0,1950,...,40.60,-76.75,PUB,59.0,0.0,0.0,0.000000,0,1,0
4,195007,24,1440,195007,24,1440,10104929,PENNSYLVANIA,42.0,1950,...,41.63,-79.68,PUB,59.0,0.0,0.0,0.000000,0,1,0


In [6]:
def get_EPA_data():
    
    ground_temp_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/temperature_fig-1.csv', skiprows=6, usecols = [0, 1], encoding='latin1')
    ground_temp_df.columns = ["YEAR", "SURFACE_TEMP_DIFF"]
    
    greenhouse_df = pd.read_csv("https://www.epa.gov/sites/default/files/2021-04/us-ghg-emissions_fig-1.csv",skiprows=6)
    greenhouse_df.columns = ["YEAR", "CO2", "METHANE", "N2O", "HFC, PFC, SF6, NF3"]
    
    precipitation_df =  pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/heavy-precip_fig-1.csv',skiprows=6)
    precipitation_df.columns = ["YEAR", "PPT_LAND_AREA", "PPT_NINE_YEAR_AVG"]
    
    sea_level_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/sea-level_fig-1.csv', skiprows=6)
    sea_level_df.columns = ["YEAR", "CSIRO_ADJ_SEA_LEVEL", "CSIRO_LOWER", "CSIRO_UPPER", "NOAA_ADJ_SEA_LEVEL"]
    sea_level_df.loc[sea_level_df['YEAR'] > 1992, 'CSIRO_ADJ_SEA_LEVEL'] = sea_level_df.loc[sea_level_df['YEAR'] > 1992, 'NOAA_ADJ_SEA_LEVEL']
    sea_level_df.drop(['CSIRO_LOWER', 'CSIRO_UPPER', 'NOAA_ADJ_SEA_LEVEL'], axis=1, inplace=True)
    
    seasonal_temp_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-04/seasonal-temperature_fig-1.csv', skiprows=6)
    seasonal_temp_df.columns = ["YEAR", "WINTER_ANOMALY", "SPRING_ANOMALY", "SUMMER_ANOMALY", "FALL_ANOMALY"]
    
    arctic_ice_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-03/arctic-sea-ice_fig-1.csv', skiprows=6)
    arctic_ice_df.columns = ["YEAR", "ICE_CVG_MARCH", "ICE_CVG_SEP"]
    
    glacier_df = pd.read_csv('https://www.epa.gov/sites/default/files/2021-03/glaciers_fig-1.csv', skiprows=6)
    glacier_df.columns = ["YEAR", "GLACIER_MASS_BAL", "NUM_OBS"]
    glacier_df.drop(['NUM_OBS'], axis=1, inplace=True)
    
    dfs = [ground_temp_df, greenhouse_df, precipitation_df, sea_level_df, seasonal_temp_df, arctic_ice_df, glacier_df]
    epa_df = reduce(lambda left, right: pd.merge(left, right, how="outer", on="YEAR"), dfs)
    epa_df = epa_df[epa_df.YEAR >= 1950]
    
    return epa_df

epa_source_df = get_EPA_data()

In [7]:
def impute_EPA_DATA(df, breaks):
    fillable_cols = df.columns[df.isnull().sum() > 0]
    for col in fillable_cols:
        temp_df = df[['YEAR', col]]
        present_df = temp_df[~ temp_df[col].isnull()]
        null_df = temp_df[temp_df[col].isnull()]
        years = sorted(np.random.choice(present_df['YEAR'], breaks))
        input_df = present_df[present_df['YEAR'].isin(years)]
        func = interpolate.interp1d(input_df['YEAR'], input_df[col], fill_value="extrapolate")
        temp_df['INTERPOLATION'] = func(temp_df['YEAR'])
        df[col] = temp_df.apply(lambda x: x['INTERPOLATION'] if isnan(x[col]) else x[col], axis=1)
    return df
        
imputed_EPA_df = impute_EPA_DATA(epa_source_df.copy(), 6)
imputed_EPA_df.head()

,YEAR,SURFACE_TEMP_DIFF,CO2,METHANE,N2O,"HFC, PFC, SF6, NF3",PPT_LAND_AREA,PPT_NINE_YEAR_AVG,CSIRO_ADJ_SEA_LEVEL,WINTER_ANOMALY,SPRING_ANOMALY,SUMMER_ANOMALY,FALL_ANOMALY,ICE_CVG_MARCH,ICE_CVG_SEP,GLACIER_MASS_BAL
49,1950,-0.63,2659.760945,1192.496732,473.537757,161.828535,0.100,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857
50,1951,-0.90,2721.103288,1182.687741,473.161684,160.274465,0.107,0.093453,3.972441,0.35,-1.24,-0.66,-1.49,6.278021,3.616373,2.060714
51,1952,0.25,2782.445631,1172.878751,472.785612,158.720395,0.077,0.099285,3.870079,0.67,-0.94,1.16,-0.63,6.269913,3.584432,1.648571
52,1953,1.35,2843.787975,1163.069761,472.409540,157.166325,0.123,0.102855,4.043307,3.02,-0.36,0.75,1.87,6.261805,3.552491,1.236429
53,1954,1.31,2905.130318,1153.260771,472.033467,155.612254,0.099,0.101262,3.929134,3.10,-0.25,0.77,1.51,6.253697,3.520550,0.824286


In [8]:
cleaned_df = pd.merge(imputed_NOAA_df, imputed_EPA_df, on='YEAR', how='inner')
cleaned_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EVENT_ID,STATE,STATE_FIPS,YEAR,...,PPT_LAND_AREA,PPT_NINE_YEAR_AVG,CSIRO_ADJ_SEA_LEVEL,WINTER_ANOMALY,SPRING_ANOMALY,SUMMER_ANOMALY,FALL_ANOMALY,ICE_CVG_MARCH,ICE_CVG_SEP,GLACIER_MASS_BAL
0,195004,28,1445,195004,28,1445,10096222,OKLAHOMA,40.0,1950,...,0.1,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857
1,195004,29,1530,195004,29,1530,10120412,TEXAS,48.0,1950,...,0.1,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857
2,195007,5,1800,195007,5,1800,10104927,PENNSYLVANIA,42.0,1950,...,0.1,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857
3,195007,5,1830,195007,5,1830,10104928,PENNSYLVANIA,42.0,1950,...,0.1,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857
4,195007,24,1440,195007,24,1440,10104929,PENNSYLVANIA,42.0,1950,...,0.1,0.085488,3.598425,0.85,-1.94,-1.68,0.32,6.286129,3.648314,2.472857


In [9]:
cleaned_df.to_pickle('Data/cleaned_data.pkl', protocol=4)